In [1]:
import sys

sys.path.append("../modelling")
# Import necessary libraries
import pandas as pd # For data manipulation and analysis
import numpy as np # For numerical operations
import pickle # For serializing and deserializing Python objects
from modelling_utils import train_knn_classifier, evaluate_model, plot_model_comparison, extract_model_features, compute_knn_indices, compute_overlap_matrix, plot_overlap_matrix, split_shuffle_data, apply_aggregation_filter
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import cohen_kappa_score


In [2]:
# Load features from a pickle file
feature_dict_path = "/home/suraj/Repositories/FM-extractors-radiomics/evaluation/features/nsclc_radiomics_stability.pkl" # Path to the pickle file containing features
with open(feature_dict_path, 'rb') as file: # Open the file in read binary mode
    data = pickle.load(file) # Load the data from the pickle file

In [24]:
# Store test accuracies for each model
results_list = []

# Iterate through each model's features
for model_name, values in data.items(): # Loop through each model and its corresponding features

    trial_0_data = [v for v in values["train"] if v["row"]["trial"] == 0]
    labels = [v["row"]["survival"] for v in trial_0_data] # Extract survival labels for the training set
    
    # Stack features
    items = np.vstack([apply_aggregation_filter(v["feature"], model_name) for v in trial_0_data]) # Stack features for the training set

    # train_items, val_items, train_labels, val_labels = train_test_split(items, labels, train_size=0.6)
    best_model, study = train_knn_classifier(items, labels, items, labels)
    # Bin the continuous predictions into many discrete categories to mimic a continuous kappa.
    # Here we use 100 bins over the interval [0, 1].      
    preds = best_model.predict_proba(items)[:, 1]
    bins = np.linspace(0, 1, 101)
    binned_preds = np.digitize(preds, bins) - 1

    for n_trial in range(1, 50):
        trial_n_data = [v for v in values["train"] if v["row"]["trial"] == n_trial]
        labels = [v["row"]["survival"] for v in trial_n_data] # Extract survival labels for the training set
        items = np.vstack([apply_aggregation_filter(v["feature"], model_name) for v in trial_n_data]) # Stack features for the training set
        trial_preds = best_model.predict_proba(items)[:, 1]

        binned_trial_preds = np.digitize(trial_preds, bins) - 1
        # Calculate the weighted Cohen's kappa score using linear weights as a continuous approximation.
        score = cohen_kappa_score(binned_preds, binned_trial_preds, weights="linear")
        row = {
            "Model": model_name,
            "Trial": n_trial,
            "AUC": score
        }
        print(row)
        results_list.append(row)

[I 2025-03-21 19:11:20,757] A new study created in memory with name: no-name-30fec52e-3fde-4146-b045-c5cd1c47a40e
[I 2025-03-21 19:11:20,779] Trial 0 finished with value: 0.6513337090260166 and parameters: {'k': 29}. Best is trial 0 with value: 0.6513337090260166.
[I 2025-03-21 19:11:20,799] Trial 1 finished with value: 0.725556494787264 and parameters: {'k': 12}. Best is trial 1 with value: 0.725556494787264.
[I 2025-03-21 19:11:20,819] Trial 2 finished with value: 0.7290199117122195 and parameters: {'k': 11}. Best is trial 2 with value: 0.7290199117122195.
[I 2025-03-21 19:11:20,841] Trial 3 finished with value: 0.6383957922419461 and parameters: {'k': 42}. Best is trial 2 with value: 0.7290199117122195.
[I 2025-03-21 19:11:20,861] Trial 4 finished with value: 0.8340377571146802 and parameters: {'k': 3}. Best is trial 4 with value: 0.8340377571146802.
[I 2025-03-21 19:11:20,881] Trial 5 finished with value: 0.6540222597914905 and parameters: {'k': 28}. Best is trial 4 with value: 0.8

[1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1.
 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1.
 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0.
 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1.
 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0.
 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1.
 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0.
 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0.
 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1.

[I 2025-03-21 19:11:27,373] A new study created in memory with name: no-name-2cb3085f-2f5b-438c-8673-e38d264854fa
[I 2025-03-21 19:11:27,384] Trial 0 finished with value: 0.6532121724429416 and parameters: {'k': 29}. Best is trial 0 with value: 0.6532121724429416.
[I 2025-03-21 19:11:27,392] Trial 1 finished with value: 0.7285385554616324 and parameters: {'k': 12}. Best is trial 1 with value: 0.7285385554616324.
[I 2025-03-21 19:11:27,399] Trial 2 finished with value: 0.7424626655395885 and parameters: {'k': 11}. Best is trial 2 with value: 0.7424626655395885.
[I 2025-03-21 19:11:27,408] Trial 3 finished with value: 0.6486099370714754 and parameters: {'k': 42}. Best is trial 2 with value: 0.7424626655395885.
[I 2025-03-21 19:11:27,416] Trial 4 finished with value: 0.8215224945994177 and parameters: {'k': 3}. Best is trial 4 with value: 0.8215224945994177.
[I 2025-03-21 19:11:27,424] Trial 5 finished with value: 0.6598102751948904 and parameters: {'k': 28}. Best is trial 4 with value: 0

[1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1.
 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1.
 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0.
 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1.
 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0.
 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1.
 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0.
 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0.
 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1.

[I 2025-03-21 19:11:27,443] Trial 7 finished with value: 0.6531652108575186 and parameters: {'k': 32}. Best is trial 4 with value: 0.8215224945994177.
[I 2025-03-21 19:11:27,451] Trial 8 finished with value: 0.6682751009674086 and parameters: {'k': 23}. Best is trial 4 with value: 0.8215224945994177.
[I 2025-03-21 19:11:27,460] Trial 9 finished with value: 0.7553888419273034 and parameters: {'k': 5}. Best is trial 4 with value: 0.8215224945994177.
[I 2025-03-21 19:11:27,469] Trial 10 finished with value: 0.657638301869071 and parameters: {'k': 34}. Best is trial 4 with value: 0.8215224945994177.
[I 2025-03-21 19:11:27,479] Trial 11 finished with value: 0.6505236216774678 and parameters: {'k': 36}. Best is trial 4 with value: 0.8215224945994177.
[I 2025-03-21 19:11:27,489] Trial 12 finished with value: 0.6620292101061332 and parameters: {'k': 27}. Best is trial 4 with value: 0.8215224945994177.
[I 2025-03-21 19:11:27,499] Trial 13 finished with value: 0.6561942331173101 and parameters: 

...

[I 2025-03-21 19:11:45,965] A new study created in memory with name: no-name-cfbfcd07-3b45-4f30-8462-e7cc84a316d6
[I 2025-03-21 19:11:45,975] Trial 0 finished with value: 0.6474711186249649 and parameters: {'k': 29}. Best is trial 0 with value: 0.6474711186249649.
[I 2025-03-21 19:11:45,984] Trial 1 finished with value: 0.7252982060674368 and parameters: {'k': 12}. Best is trial 1 with value: 0.7252982060674368.
[I 2025-03-21 19:11:45,993] Trial 2 finished with value: 0.7381187188879497 and parameters: {'k': 11}. Best is trial 2 with value: 0.7381187188879497.
[I 2025-03-21 19:11:46,003] Trial 3 finished with value: 0.6413661125199587 and parameters: {'k': 42}. Best is trial 2 with value: 0.7381187188879497.
[I 2025-03-21 19:11:46,012] Trial 4 finished with value: 0.8310674368366676 and parameters: {'k': 3}. Best is trial 4 with value: 0.8310674368366676.
[I 2025-03-21 19:11:46,020] Trial 5 finished with value: 0.648480792711562 and parameters: {'k': 28}. Best is trial 4 with value: 0.

[1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1.
 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1.
 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0.
 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1.
 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0.
 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1.
 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0.
 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0.
 0. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1.

[I 2025-03-21 19:11:51,544] A new study created in memory with name: no-name-779880b1-429e-4acd-af56-4ccf7015ed05
[I 2025-03-21 19:11:51,563] Trial 0 finished with value: 0.5883464825772519 and parameters: {'k': 29}. Best is trial 0 with value: 0.5883464825772519.
[I 2025-03-21 19:11:51,579] Trial 1 finished with value: 0.6529304029304029 and parameters: {'k': 12}. Best is trial 1 with value: 0.6529304029304029.
[I 2025-03-21 19:11:51,594] Trial 2 finished with value: 0.6650347515732131 and parameters: {'k': 11}. Best is trial 2 with value: 0.6650347515732131.
[I 2025-03-21 19:11:51,611] Trial 3 finished with value: 0.5974218089602704 and parameters: {'k': 42}. Best is trial 2 with value: 0.6650347515732131.
[I 2025-03-21 19:11:51,627] Trial 4 finished with value: 0.6794284775054006 and parameters: {'k': 3}. Best is trial 4 with value: 0.6794284775054006.
[I 2025-03-21 19:11:51,644] Trial 5 finished with value: 0.5873133276979432 and parameters: {'k': 28}. Best is trial 4 with value: 0

[0.33333333 0.33333333 0.33333333 0.         0.         0.
 0.         0.33333333 0.         0.66666667 0.33333333 0.
 0.33333333 0.33333333 0.33333333 0.         0.33333333 0.33333333
 0.33333333 0.         0.66666667 0.33333333 0.         0.33333333
 1.         0.33333333 0.66666667 0.33333333 0.33333333 0.
 1.         0.         0.33333333 0.33333333 0.33333333 0.33333333
 0.         0.         0.66666667 0.33333333 0.         0.33333333
 0.66666667 0.         0.66666667 0.         0.66666667 0.33333333
 0.33333333 0.33333333 0.         0.         0.66666667 0.33333333
 0.         1.         0.         0.33333333 0.33333333 0.
 0.66666667 0.33333333 0.33333333 0.         0.33333333 0.
 0.33333333 0.33333333 0.         0.         0.66666667 0.
 0.33333333 0.33333333 1.         0.66666667 0.         0.66666667
 0.         0.66666667 0.33333333 0.33333333 0.33333333 0.66666667
 0.66666667 0.66666667 1.         0.66666667 0.         0.33333333
 0.66666667 0.66666667 0.33333333 0.       

KeyboardInterrupt: 

In [14]:
results_df = pd.DataFrame(results_list)
results_df

,Model,Trial,AUC
0,FMCIBExtractor,1,0.613424
1,FMCIBExtractor,2,0.623333
2,FMCIBExtractor,3,0.591822
3,FMCIBExtractor,4,0.611616
4,FMCIBExtractor,5,0.627019
...,...,...,...
485,ModelsGenExtractor,45,0.605887
486,ModelsGenExtractor,46,0.603210
487,ModelsGenExtractor,47,0.626468
488,ModelsGenExtractor,48,0.629602


In [ ]:
import plotly.express as px

# Create a box plot without mapping colors to "Model" to ensure all boxes use the same color.
fig = px.box(results_df, x="Model", y="AUC",
             title="AUC Distribution Across Trials by Model",
             template="simple_white")

# Update traces to ensure every box has the same fill color.
fig.update_traces(marker=dict(color="#1f77b4", size=8, line=dict(width=1, color="black")),
                  boxmean=True)

fig.show()
